In [1]:
import sys
sys.path.append('/home/zhw272/programming/preprocessing-pipeline/in_development/Will')
from cell_extractor.CellDetectorBase import CellDetectorBase
from abakit.lib.annotation_layer import Cell,random_string
import numpy as np
from random import sample
from abakit.lib.UrlGenerator import UrlGenerator

def numpy_to_json(data,color_hex = None):
    cells = []
    for coord in data:
        cell = Cell(np.array(coord,dtype=float),random_string())
        cell.category = 'Round3'
        del cell.description
        cell_json = cell.to_json()
        if color_hex is not None:
            cell_json["props"] = [color_hex]
        cells.append(cell_json)
    return cells

In [4]:
animal = 'DK41'
base = CellDetectorBase(animal,round = 2)
detections = base.load_detections()
sure = detections[detections.predictions==2]
unsure = detections[detections.predictions==0]
len(sure),len(unsure),len(detections)

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'histology.scene_order' in 'field list'")
[SQL: SELECT histology.created AS histology_created, histology.active AS histology_active, histology.id AS histology_id, histology.prep_id AS histology_prep_id, histology.virus_id AS histology_virus_id, histology.label_id AS histology_label_id, histology.performance_center AS histology_performance_center, histology.anesthesia AS histology_anesthesia, histology.perfusion_age_in_days AS histology_perfusion_age_in_days, histology.perfusion_date AS histology_perfusion_date, histology.exsangination_method AS histology_exsangination_method, histology.fixative_method AS histology_fixative_method, histology.special_perfusion_notes AS histology_special_perfusion_notes, histology.post_fixation_period AS histology_post_fixation_period, histology.whole_brain AS histology_whole_brain, histology.block AS histology_block, histology.date_sectioned AS histology_date_sectioned, histology.sid

In [3]:
sure_data = sure[['col','row','section']].sample(500).sort_values('section').to_numpy().tolist()
unsure_data = unsure[['col','row','section']].sample(831).sort_values('section').to_numpy().tolist()
sure_cells = numpy_to_json(sure_data)
unsure_cells = numpy_to_json(unsure_data,color_hex='#1d66db')
urlGen = UrlGenerator()
image_layer = 'precomputed://https://activebrainatlas.ucsd.edu/data/'+animal+'/neuroglancer_data/'
urlGen.add_stack_image(animal,channel=1)
urlGen.add_stack_image(animal,channel=2,color='red')
urlGen.add_stack_image(animal,channel=3,color='green')
urlGen.add_annotation_layer('Sure',annotations = sure_cells)
urlGen.add_annotation_layer('Unsure',annotations = unsure_cells)
urlGen.add_to_database('test QC3',34)

NameError: name 'sure' is not defined

In [ ]:
from abakit.lib.Controllers.SqlController import SqlController

controller = SqlController('DK52')
controller.get_resolution('DK52')

array([ 0.325,  0.325, 20.   ])